In [19]:
import numpy as np
from sdccode.modules import bayesianL
from sdccode.modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

def cubicarray(MM, pr=False):
    if pr: print('numero di elementi: ', len(MM))
    # In input bisogna fornire la lista dei monomi M. Ogni elemento della lista M e'
    # una lista [a,b,c] tale per cui M_i=x^{2a}y^{2b}z^{2c} e a+b+c=n, dove n e' il grado del monomio.
    # I monomi in input comprendono tutte le permutazioni.
    # Voglio ora applicare le permutazioni x,y,z in modo da simmetrizzare i monomi e ottenere le armoniche cubiche.
    lenM = len(MM)
    M=[]
    #Converto M a una lista di liste
    for i in range(len(MM)):
        M.append(list(MM[i]))
    Mt = copy.copy(M)
    # devo prima capire nella base dei monomi quali sono legati da permutazione. posso nominarli.
    nameC = []
    conta = 0
    contaci = 0
    if pr: print('numero cicli teornicamente necessari', int(len(M) ** 2 * 3 / 2))
    for i in range(lenM):
        rep = M[i]
        # rep (M[i]) e' il mio rappresentante nella lista,
        # voglio vedere quali altri elementi sono equivalenti per permutazione a rep
        nameC_i = []
        if not (rep in Mt): continue
        # se rep non sta nella lista temporanea Mt significa che e' equivalente a un elemento precedente: salto il ciclo
        nameC_i.append(np.array(rep))
        # in nameC_i inserisco gli elementi equivalenti per simmetria a rep
        for j in range(i + 1, lenM):
            # parto a cercare dall'elemento successivo nella lista, gli altri li ho gia' sicuramente controllati.
            Mjt = list(copy.copy(M[j]))
        
            if rep[0] in Mjt:
                Mjt.remove(rep[0])
                if rep[1] in Mjt:
                    Mjt.remove(rep[1])
                    if rep[2] in Mjt:
                        # rep e M[j] sono simmetrici per permutazione!
                        nameC_i.append(np.array(M[j]))
                        Mt.remove(M[j])
                        
                        # rimuovo da Mt l'elemento M[j] che e' simmetrico per permutazione a rep
                        conta += 1
        contaci += len(nameC_i)
        nameC.append(nameC_i)
        # aggiungo la lista degli elementi associati a M[i], nameC_i, alla lista degli elementi NON equivalenti, nameC!
    if contaci != len(M): raise ValueError
    if pr: print('numero cicli compiuti', conta)
    if pr: print('numero di elementi non equivalenti per simmetria cubica: ', len(nameC))
    # in output c'e' la lista degli elementi che formano una classe di equivalenza.
    return nameC

def datadicG(data, sigmadata, G):

    dicdata = {}
    
    dicsigma = {}
    
    for i in range(len(G)):
        dicdata[tuple(G[i])] = data[i]
        dicsigma[tuple(G[i])] = sigmadata[i]
        
    return dicdata, dicsigma

def datiplot(C, dic, dics):
    gplot=[]
    dataplot=[]
    datasigmaplot=[]
    for i in range(len(C)):
        gplot.append(np.linalg.norm(C[i][0]))
        dataav=0
        datasigma=0
        for j in range(len(C[i])):
            dataav+=dic[tuple(C[i][j])]/len(C[i])  
            datasigma+=(dics[tuple(C[i][j])]/len(C[i])  )**2
        dataplot.append(dataav)
        datasigmaplot.append(np.sqrt(datasigma))
    return np.array(gplot), np.array(dataplot), np.array(datasigmaplot)



In [20]:
data=np.transpose(np.loadtxt('nlttk.out'))
sdata, grid = bayesianL.generatesorteddata(data, 500)

[0. 0. 1.] [0. 1. 0.] [1. 0. 0.]


In [21]:
mN, SN, y_infer, sy_infer=bayesianL.bayesianpol(grid,sdata,7,70,0.00001,grid,True)

parametri ottimali [ 8.55660905e-01 -2.43249610e+01  7.73733404e+01 -7.40904097e+02
  2.79165799e+01 -1.05581423e+03 -9.63653596e+01  2.83340052e+00
  2.16499698e+02  4.35943202e+01 -8.42983010e+00  7.31869623e+01
 -1.83876044e+00  9.71124360e-01 -1.42868508e+01 -1.27322316e+00]
numero di armoniche cubiche 8
valore a 0 inferito 0.6753076948017609 bias 0.8556609048407888 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche : 436169543166.93066


In [22]:
N=499
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=bayesianL.bestfit(grid, sdata, N, grid, False, True)

parametri ottimali [ 5.43509474e-01 -6.21021003e+01 -4.01430036e+01  1.89284004e+03
  9.89884414e+02 -1.24922384e+04 -3.71647210e+02 -8.31600096e+02
 -1.22124616e+03 -6.54823615e+02  7.02791201e+02  4.74587024e+03
  6.97818787e+02  2.13240718e+02 -5.73515435e+02  6.10305857e+01
 -1.72711522e+03 -2.09971613e+03 -6.91601414e+02  1.57096081e+03
 -1.13298908e+03  3.22071738e+02  6.61070998e+02  2.45287656e+03
  3.85924322e+02 -4.98313862e+02 -1.08014666e+03 -2.18948681e+02
 -2.53611456e+03  1.45256496e+03 -5.70051719e+02  2.99533051e+03
 -1.56223933e+03  6.92882205e+02 -9.49337276e+02  4.73526310e+02
 -2.24958676e+02]
numero di armoniche cubiche 24
valore a 0 inferito 0.5032056315847729 bias 0.543509473533728 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche : 1.9250384287318537e-21
grado ottimale 28 grado massimo tentato 28


In [14]:
%matplotlib widget
plt.plot(log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
C=cubicarray(list(grid), True)
dic, dics=datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=datiplot(C, dic, dics)
C=cubicarray(grid[:N], True)
dic, dics=datadicG(y_infer, np.sqrt(sy_infer), grid[:N])
gplotfit, dataplotfit, datasigmaplotfit=datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(gplotfit, dataplotfit+datasigmaplotfit, dataplotfit-datasigmaplotfit,  alpha=0.3, color='red')
plt.plot(gplotfit, dataplotfit, color='red')
plt.errorbar(gplot, dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118
numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [16]:
mr, sr, logevidence= bayesianL.bayesianmodelprediction(grid, sdata, N, grid, False, True)

model average prediction 0.3774125870564994 0.012359456073076116
